## Natural language inference

### Contradictory dear watson
***

Project: https://www.kaggle.com/competitions/contradictory-my-dear-watson

### Starter guide

https://keras.io/guides/keras_nlp/getting_started/

#### Importar liberias

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

E0000 00:00:1728597865.876432      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1010 22:04:25.884490946      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1010 22:04:25.884504274      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1010 22:04:25.884507373      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1010 22:04:25.884509726      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1010 22:04:25.884512093      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

TensorFlow version: 2.16.1
KerasNLP version: 0.14.4


#### Conf

In [ ]:
try:
    # detect and init the TPU
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
except ValueError:
    strategy = tf.distribute.get_strategy()  # default strategy if no TPU available

In [4]:
RESULT_DICT = {
    0 : "entailment",
    1 : "neutral",
    2 : "contradiction"
}

#### Data

In [5]:
df_train_data = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
df_submit_data = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

In [6]:
df_train, df_test = train_test_split(df_train_data, test_size=0.2, random_state=42)

### Intermediate model

**Fine tuning a pretrained BERT backbone**

#### Data prep

In [7]:
# Asegúrate de que 'premise' y 'hypothesis' sean listas de strings
premises = df_train['premise'].astype(str).tolist()
hypothesis = df_train['hypothesis'].astype(str).tolist()

# Define las labels (etiquetas) para entrenar
labels = tf.convert_to_tensor(df_train['label'])

# Verifica que todas las entradas tengan la misma longitud
assert len(premises) == len(hypothesis) == len(labels), "Las dimensiones de las entradas no coinciden."

# Crea un dataset a partir de las entradas
train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypothesis), labels)).batch(32)


#### Define model

In [8]:
# Define el modelo dentro del scope de la estrategia
with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi",
        num_classes=3
    )
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

I0000 00:00:1728597905.465018      13 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


#### Train model

In [9]:
classifier.fit(train_dataset, epochs=10)

Epoch 1/10


2024-10-10 22:06:09.572401: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1728597971.994439     818 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(1bdbb808d4e4ff8f:0:0), session_name()
I0000 00:00:1728598070.103509     818 tpu_compile_op_common.cc:245] Compilation of 1bdbb808d4e4ff8f:0:0 with session name  took 1m38.109023089s and succeeded
I0000 00:00:1728598070.392543     818 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(1bdbb808d4e4ff8f:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_4238798980000106984", property.function_library_fingerprint = 16587923574941677658, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().

303/303 ━━━━━━━━━━━━━━━━━━━━ 157s 137ms/step - accuracy: 0.3333 - loss: 1.1635
Epoch 2/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 138ms/step - accuracy: 0.3313 - loss: 1.1470
Epoch 3/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 139ms/step - accuracy: 0.3321 - loss: 1.1462
Epoch 4/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 137ms/step - accuracy: 0.3389 - loss: 1.1454
Epoch 5/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 137ms/step - accuracy: 0.3047 - loss: 1.1551
Epoch 6/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 137ms/step - accuracy: 0.3392 - loss: 1.1351
Epoch 7/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 138ms/step - accuracy: 0.3309 - loss: 1.1345
Epoch 8/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 138ms/step - accuracy: 0.3155 - loss: 1.1312
Epoch 9/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 137ms/step - accuracy: 0.3039 - loss: 1.1346
Epoch 10/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 138ms/step - accuracy: 0.3270 - loss: 1.1278


### Advanced model

**Fine tuning with user-controlled preprocessing**

#### Data prep

Separate preprocessing from the same preset

Each model architecture has a parallel preprocessor Layer with its own from_preset constructor. Using the same preset for this Layer will return the matching preprocessor as the task.

In [36]:
# Asegúrate de que 'premise' y 'hypothesis' sean listas de strings
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()
labels = tf.convert_to_tensor(df_train['label'])

assert len(premises) == len(hypotheses) == len(labels), "Las dimensiones de las entradas no coinciden."

train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels))

In [37]:
sample_premise = premises[0]  # Usa un ejemplo de tu lista
sample_hypothesis = hypotheses[0]

In [42]:
# Preprocesar un solo ejemplo
output = preprocessor(sample_premise, sample_hypothesis)

# Extraer los valores
input_ids = output[0]['token_ids'].numpy()  # Convierte el tensor a un array de NumPy
segment_ids = output[0]['segment_ids'].numpy()  # Convierte el tensor a un array de NumPy
attention_mask = output[0]['padding_mask'].numpy()  # Convierte el tensor a un array de NumPy

# Imprimir los resultados
print("Input IDs:", input_ids)
print("Segment IDs:", segment_ids)
print("Attention Mask:", attention_mask)


Input IDs: [  101 76295   763 28089 10429   770 25473 11242 19249 16152   119   102
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0    

**Preprocesador Bert**

In [33]:
train_cached = (
    train_dataset.map(lambda x, y: (preprocessor(x[0], x[1]), y), num_parallel_calls=tf.data.AUTOTUNE)
                  .cache()
                  .prefetch(tf.data.AUTOTUNE)
)


#### Define model


- tf.keras.optimizers.Adam(): Este es un optimizador que se adapta durante el entrenamiento y es popular por su rendimiento en muchos problemas de aprendizaje profundo.


- tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True): Esta función se utiliza para medir la discrepancia entre las predicciones del modelo y las etiquetas verdaderas. El parámetro from_logits=True indica que las salidas del modelo son logits (valores sin pasar por una función de activación como softmax), lo cual es común en problemas de clasificación.


- metrics=['accuracy']: Esto permite que el modelo calcule y muestre la precisión durante el entrenamiento y la validación. Es una medida común de rendimiento en problemas de clasificación.

In [35]:
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
5289,4e5ad9e03a,اب اتنا خفیہ تھا یہ.,یہ عوامی معلومات تھی۔,ur,Urdu,2
6647,fea0d3c7e8,oh that's accommodating,That is disruptive.,en,English,2
1245,4ce586b61e,more than anything else in this day and age th...,"In your decisions age is a big factor, and I a...",en,English,1
4417,0dbcd1012b,"Aie! les boucaniers en-dessous s'écriaient, et...",Les Buccaneers étaient bruyants quand ils disa...,fr,French,0
5662,157895e59c,"一个采购战略组织，是壮大的力资本发展战略的一部分, 这是在原则VI上讨论的。",原则四涉及财富500强机构的资本发展战略。,zh,Chinese,1


In [44]:
import tensorflow as tf
import keras_nlp
import numpy as np

# Preparar los datos
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()
labels = tf.convert_to_tensor(df_train['label'].tolist())

# Definir el preprocesador
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_base_multi",
    sequence_length=512,
)

# Procesar los textos y obtener los IDs, las máscaras de atención y los IDs de segmento
def preprocess_data(premise, hypothesis):
    # Usar el preprocesador
    outputs = preprocessor(premise, hypothesis)
    
    # Convertir a NumPy para manejar como arrays
    input_ids = np.array(outputs[0])  # Primer elemento de la tupla
    attention_mask = np.array(outputs[1])  # Segundo elemento de la tupla
    segment_ids = np.array(outputs[2])  # Tercer elemento de la tupla

    return (input_ids, attention_mask, segment_ids)

# Crear el dataset
train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels))

# Preprocesar los datos y generar el dataset final
train_cached = (
    train_dataset.map(lambda x, y: (preprocess_data(x[0], x[1]), y), num_parallel_calls=tf.data.AUTOTUNE)
                  .cache()
                  .prefetch(tf.data.AUTOTUNE)
)

# Usar strategy.scope() si estás utilizando estrategia de distribución
with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi",
        preprocessor=None,  # Ya manejamos el preprocesador antes
        num_classes=3  # Asegúrate de que esto coincide con tus datos
    )

    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    # Entrenar el modelo
    classifier.fit(
        train_cached,
        epochs=3,
    )


NotImplementedError: in user code:

    File "/tmp/ipykernel_13/3902955835.py", line 33, in None  *
        lambda x, y: (preprocess_data(x[0], x[1]), y)
    File "/tmp/ipykernel_13/1183522696.py", line 23, in preprocess_data  *
        attention_mask = np.array(outputs[1])  # Segundo elemento de la tupla

    NotImplementedError: Cannot convert a symbolic tf.Tensor (args_1:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.


In [25]:
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()
labels = tf.convert_to_tensor(df_train['label'])

train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels))

# Preprocesar los datos
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_multi", sequence_length=512)

In [26]:
# Aplicar el preprocesador a cada ejemplo en el conjunto de datos
train_cached = (
    train_dataset.map(lambda x, y: preprocessor(x[0], x[1]), num_parallel_calls=tf.data.AUTOTUNE)
                  .cache()
                  .prefetch(tf.data.AUTOTUNE)
)


In [28]:

# Usar strategy.scope() si estás utilizando estrategia de distribución
with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi",
        preprocessor=None,
        num_classes=3  # Asegúrate de que esto coincide con tus datos
    )

    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    # Entrenar el modelo
    classifier.fit(
        train_cached,
        epochs=3,
    )

IndexError: list index out of range

In [27]:
for example in train_dataset.take(1):
    print(example)


((<tf.Tensor: shape=(), dtype=string, numpy=b'\xd8\xa7\xd8\xa8 \xd8\xa7\xd8\xaa\xd9\x86\xd8\xa7 \xd8\xae\xd9\x81\xdb\x8c\xdb\x81 \xd8\xaa\xda\xbe\xd8\xa7 \xdb\x8c\xdb\x81.'>, <tf.Tensor: shape=(), dtype=string, numpy=b'\xdb\x8c\xdb\x81 \xd8\xb9\xd9\x88\xd8\xa7\xd9\x85\xdb\x8c \xd9\x85\xd8\xb9\xd9\x84\xd9\x88\xd9\x85\xd8\xa7\xd8\xaa \xd8\xaa\xda\xbe\xdb\x8c\xdb\x94'>), <tf.Tensor: shape=(), dtype=int64, numpy=2>)


In [24]:
import tensorflow as tf
import keras_nlp

# Prepara tus datos (asegúrate de que esto esté hecho antes de la compilación)
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()
labels = tf.convert_to_tensor(df_train['label'])

train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels))

# Preprocesar los datos
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_multi", sequence_length=512)
train_cached = (
    train_dataset.map(preprocessor, num_parallel_calls=tf.data.AUTOTUNE)
                  .cache()
                  .prefetch(tf.data.AUTOTUNE)
)

# Usar strategy.scope() si estás utilizando estrategia de distribución
with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi",
        preprocessor=None,
        num_classes=3  # Asegúrate de que esto coincide con tus datos
    )

    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    # Entrenar el modelo
    classifier.fit(
        train_cached,
        epochs=3,
    )


IndexError: list index out of range

In [17]:
with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi", 
        preprocessor = None, 
        num_classes=3  # Cambia esto según tus datos
    )
    
    classifier.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

    classifier.fit(
    train_cached,
    epochs=3,
)

IndexError: list index out of range

#### Train model

In [ ]:
classifier.fit(
    train_cached,
    epochs=3,
)

In [ ]:
# Asegúrate de que 'premise' y 'hypothesis' sean listas de strings
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()
labels = tf.convert_to_tensor(df_train['label'])

assert len(premises) == len(hypotheses) == len(labels), "Las dimensiones de las entradas no coinciden."

train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels))

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_base_multi",
    sequence_length=512,
)

train_cached = (
    train_dataset
    .map(lambda x, y: (
        preprocessor(x[0]), 
        preprocessor(x[1]),
        y
    ), num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi", 
        num_classes=3
    )
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )




In [ ]:
classifier.fit(
    train_cached,
    epochs=3,
)

In [ ]:
# Asegúrate de que 'premise' y 'hypothesis' sean listas de strings
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()
labels = tf.convert_to_tensor(df_train['label'])

assert len(premises) == len(hypotheses) == len(labels), "Las dimensiones de las entradas no coinciden."

train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels))

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_base_multi",
    sequence_length=512,
)

train_cached = (
    train_dataset
    .map(lambda x, y: (
        preprocessor(x[0]),  # premisas
        preprocessor(x[1]),  # hipótesis
        y  # etiquetas
    ), num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi", 
        num_classes=3
    )
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )



In [ ]:
print(type(x[0]), x[0])  # Debe ser una cadena
print(type(x[1]), x[1])  # Debe ser una cadena


In [ ]:
classifier.fit(
    train_cached,
    epochs=3,
)


SyntaxError: invalid syntax (579925521.py, line 3)

In [ ]:
#### Data prep b

Custom preprocessing

In cases where custom preprocessing is required, we offer direct access to the Tokenizer class that maps raw strings to tokens. It also has a from_preset() constructor to get the vocabulary matching pretraining